In [2]:
'''
This is a script to build a version of the analysis flash response dataframe,
but using a cells image response pca to look at within cell variace, rather 
than the simple mean response. This seemed like a nice idea because it would 
normalize within-cell responces, but it didn't seem to make a difference in practice
so we never used in :)
Created by Yoni Browning, August 2018
'''
%load_ext autoreload
%autoreload 2
import matplotlib as mpl
%matplotlib inline
import scipy as sp
import numpy as np
import pandas as pd
# sometimes order maters on these
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import oBehave.plotting_stuff as obps
import oBehave.helperfunctions as obhf
import oBehave.neural_analysis as obna
import oBehave.behavior_analysis as obba
import sklearn.decomposition as decomp


from visual_behavior.ophys.dataset.visual_behavior_ophys_dataset import VisualBehaviorOphysDataset
from visual_behavior.ophys.response_analysis.utilities import get_trace_around_timepoint,get_nearest_frame
from visual_behavior.ophys.response_analysis.response_analysis import ResponseAnalysis



In [9]:
manifest = obhf.load_manifest()
manifest.head()


,experiment_id,experiment_container_id,targeted_structure,imaging_depth,cre_line,reporter_line,full_genotype,session_type,donor_id,specimen_id,experiment_date
0,644942849,644919226,VISp,175,Vip-IRES-Cre,Ai148,Vip-IRES-Cre;Ai148,behavior_session_A,329071,596175193,10/24/2017 10:32
1,645035903,644919226,VISp,175,Vip-IRES-Cre,Ai148,Vip-IRES-Cre;Ai148,behavior_session_B,329071,596175193,10/25/2017 10:34
2,645086795,644919226,VISp,175,Vip-IRES-Cre,Ai148,Vip-IRES-Cre;Ai148,behavior_session_C,329071,596175193,10/26/2017 10:27
3,645362806,644919226,VISp,175,Vip-IRES-Cre,Ai148,Vip-IRES-Cre;Ai148,behavior_session_D,329071,596175193,10/30/2017 11:41
4,646922970,646555436,VISp,175,Vip-IRES-Cre,Ai148,Vip-IRES-Cre;Ai148,behavior_session_A,330982,596393048,11/7/2017 14:19


In [ ]:
keylist = ['experiment_id','cell_indicies','cell_specimen_ids','dff_image_pca','evt_image_pca']
cell_data = {k:[] for k in keylist}

for ii,row in manifest.iterrows():
    experiment_id = row['experiment_id']
    print(experiment_id)
    dataset= VisualBehaviorOphysDataset(experiment_id=experiment_id, cache_dir=obhf.drive_path)
    tme = dataset.timestamps_ophys
    start_times = dataset.stimulus_table.start_time
    events = obhf.load_dff_events_file(experiment_id=experiment_id)
    for cell_index,cell_spec in zip(dataset.cell_indices,dataset.cell_specimen_ids):
        dff = dataset.dff_traces[cell_index,:]
        evt = events[cell_index,:]
        _,pca_dff = obna.singlecellpca(dff,tme,start_times)
        _,pca_evt = obna.singlecellpca(evt,tme,start_times)
        # Save Data
        cell_data['experiment_id'].append(experiment_id)
        cell_data['cell_indicies'].append(cell_index)
        cell_data['cell_specimen_ids'].append(cell_spec)
        cell_data['dff_image_pca'].append(pca_dff)
        cell_data['evt_image_pca'].append(pca_evt)
CellData = pd.DataFrame(CellData)
CellData.to_pickle('./ImagePCADataFrame.pkl')


644942849
645035903
645086795
645362806
